A continuación se muestra una descripción de los módulos de `pre-procesamiento` y `feature engineering`. El objetivo de dichas clases se centra en preparar nuestro dataset para su posterior análisis.

---

## Clase `FeatureEngineering`

### Objetivo:
Esta clase está diseñada para realizar tareas de `feature engineering`, relacionadas con resúmenes de texto. Incluye funciones para calcular distintas métricas y rasgos con el objetivo de identificar patrones característicos de resúmenes de calidad.

### Elementos Principales:

#### 1. `__init__`
- Inicializa la clase `FeatureEngineering`.

#### 2. `fe_pipeline`
- Esta función pipeline coordina la aplicación de varias métricas.
- Utiliza diferentes métodos basados en el parámetro `method` especificado.
- Acepta un dataframe, tipo de método y un encoder para el `SentenceTransformer` (métrica de similitud).
- Devuelve el dataframe modificado con columnas de métricas y rasgos añadidas.

#### Cálculo de Métricas y Rasgos:

##### A. Métricas de Superposición de Palabras (WOMs por sus siglas en inglés):
- **Métrica ROUGE (`add_rouge_scores`):** Añade las métricas de puntuación ROUGE a los datos.
- **Métrica BLEU (`add_bleu_scores`):** Añade las métricas de puntuación BLEU con un método de suavizado específico.

##### B. Rasgos de Puntuación Basadas en Gramática (GBSMs por sus siglas en inglés):
- **Rasgos de Legibilidad (`add_readability_scores`):** Añade varios rasgos de legibilidad como Flesch Reading Ease, Recuento de Sílabas y Recuento de Léxico.

##### C. Métricas de Evaluación Intrínseca:
- **Métricas sin Referencia (`add_reference_free_scores`):** Añade métricas sin referencia como la relación de compresión y la Similitud de Jaccard. Estás métricas son libres de resúmenes de referencias. Solo se comparan contra el texto de entrada.

##### D. Métricas de Similitud Textual:
- **Similitud Textual (`add_textual_similarity_scores`):** Añade métricas de similitud textual calculadas a partir de embeddings de de modelos pre-entrenados `SentenceTransformer`.

##### E. Funciones de Utilidad:
- **Adición de Resumen de Referencia (`add_ref_summary`):** Se utiliza para añadir un resumen de referencia al dataframe.
- **Cálculos del embedding del texto (`compute_sentence_embeddigns`):** Calcula los embeddings del texto usando un modelo especificado que se pasa como argumento.

### Notas:
- Cada función generalmente toma un dataframe y un método como argumentos, y puede tener otros requisitos específicos como nombres de columnas o un codificador SentenceTransformer.
- La clase utiliza bibliotecas como `pandas`, `SentenceTransformer` y posiblemente otras como `Rouge` para sus operaciones.

---

# Clase `Preprocessor`

### Objetivo:
Esta clase está diseñada para el pre-procesado. Los diferentes métodos están destinados a manejar tareas como eliminar espacios en cadenas de texto, manejar valores faltantes, realizar reemplazos con expresiones regulares y hacer cambios en el `dtype` de las columnas .

### Elementos Principales:

#### 1. `__init__`
- Inicializa la clase `Preprocessor`.

#### 2. `preprocessing_pipeline`
- El pipeline central que aplica una serie de pasos de preprocesamiento.
- Incluye transformaciones como la conversión de columnas de objeto a booleano, reemplazos con expresiones regulares, eliminación de espacios en cadenas de texto y manejo de valores faltantes.
- Acepta un dataframe y devuelve el dataframe procesado.

#### Métodos de Preprocesamiento Específicos:

##### A. `strip_str_columns`
- Elimina espacios blancos de las columnas de texto.

##### B. `object_to_bool`
- Convierte columnas de tipo objeto con valores booleanos a columnas de tipo booleano.

##### C. `regex_replacements`
- Reemplaza patrones en el formato `[números]` con una cadena vacía en las columnas de texto. Estos número han sido introducidos en el dataset como anotación extra. Se eliminan porque no tienen que ver con el texto original.

##### D. `handle_missing_values`
- Maneja los valores faltantes en el dataframe.
- Rellena los valores `None` con `NaN`, reemplaza las cadenas vacías con `NaN` y elimina las filas sin valores clave en ciertas columnas (por ejemplo, aquellas que contienen "title", "text" o "summary").

### Notas:
- La clase utiliza bibliotecas como `pandas` y `numpy` para sus operaciones.
- Cada función generalmente toma un dataframe y devuelve una versión modificada del dataframe después de aplicar transformaciones específicas.

---

In [1]:
import time

import numpy as np
import pandas as pd

from tqdm.notebook import tqdm 
from datasets import load_dataset

from sentence_transformers import SentenceTransformer

import seaborn as sns
sns.set(style='darkgrid')
sns.set_palette(sns.color_palette('husl', 8))

import matplotlib.pyplot as plt
%matplotlib inline

params = {
    'legend.fontsize': 'x-large',
    'figure.figsize': (16, 5),
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
}
plt.rcParams.update(params)

from src.loading import huggingface_dataset_to_dataframes
from src.preprocessing import Preprocessor
from src.feature_engineering import FeatureEngineering

from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# 1. Load human learning datasets
test_axis, validation_axis = (
    pd.read_pickle("data/raw/test_axis.pkl"),
    pd.read_pickle("data/raw/validation_axis.pkl"),
)
train_comparisons, validation_comparisons = (
    pd.read_pickle("data/raw/train_comparisons.pkl"),
    pd.read_pickle("data/raw/validation_comparisons.pkl"),
)

In [4]:
# 2. Pre-processing
preprocessor = Preprocessor()

train_comparisons = preprocessor.preprocessing_pipeline(train_comparisons)
validation_comparisons = preprocessor.preprocessing_pipeline(validation_comparisons)

test_axis = preprocessor.preprocessing_pipeline(test_axis)
validation_axis = preprocessor.preprocessing_pipeline(validation_axis)

In [5]:
start_time = time.time()

# 3. Feature engineering
fe = FeatureEngineering()
encoder = SentenceTransformer('all-MiniLM-L6-v2')

train_comparisons = fe.fe_pipeline(train_comparisons, "comparisons", encoder) 
validation_comparisons = fe.fe_pipeline(validation_comparisons, "comparisons", encoder)
                                                
test_axis = fe.fe_pipeline(test_axis, "axis", encoder) 
validation_axis = fe.fe_pipeline(validation_axis, "axis", encoder)

print("--- %s seconds ---" % (time.time() - start_time))

C:\Users\Usuario\anaconda3\lib\site-packages\huggingface_hub\file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


Batches:   0%|          | 0/1451 [00:00<?, ?it/s]

Batches:   0%|          | 0/1183 [00:00<?, ?it/s]

Batches:   0%|          | 0/1451 [00:00<?, ?it/s]

Batches:   0%|          | 0/1451 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/99 [00:00<?, ?it/s]

Batches:   0%|          | 0/135 [00:00<?, ?it/s]

Batches:   0%|          | 0/134 [00:00<?, ?it/s]

Batches:   0%|          | 0/135 [00:00<?, ?it/s]

--- 2295.34489774704 seconds ---


In [10]:
# 4. Store data
store = False

if store:

    train_comparisons.to_pickle("data/processed/fe_train_comparisons.pkl")
    validation_comparisons.to_pickle("data/processed/fe_validation_comparisons.pkl")

    test_axis.to_pickle("data/processed/fe_test_axis.pkl")
    validation_axis.to_pickle("data/processed/fe_validation_axis.pkl")

In [11]:
train_comparisons.head()

,worker,batch,split,id,source,subsource,title,text,summary_0,policy_0,note_0,summary_1,policy_1,note_1,confidence,choice,ref_summary,m0_rouge_1_f,m0_rouge_2_f,m0_rouge_l_f,m1_rouge_1_f,m1_rouge_2_f,m1_rouge_l_f,m0_bleu,m1_bleu,m0_flesch_reading_ease,m0_syllable_count,m0_lexicon_count,m0_sentence_count,m0_char_count,m0_letter_count,m0_polysyllab_count,m0_monosyllab_count,m1_flesch_reading_ease,m1_syllable_count,m1_lexicon_count,m1_sentence_count,m1_char_count,m1_letter_count,m1_polysyllab_count,m1_monosyllab_count,m0_compression_ratio,m0_jaccard_similarity_1,m0_jaccard_similarity_2,m1_compression_ratio,m1_jaccard_similarity_1,m1_jaccard_similarity_2,m0_text_summary_xfmr_similarity,m1_text_summary_xfmr_similarity,m0_ref_summary_xfmr_similarity,m1_ref_summary_xfmr_similarity
0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,Mum is mad at me for not flying on my own trip...,sup1,NaN,I have made sure my mother is comfortable with...,sup1,NaN,NaN,1,mum isn't speaking to me because I booked a fl...,0.363636,0.181818,0.303030,0.368421,0.100000,0.210526,0.057345,0.046340,97.54,17.0,16.0,1.0,53.0,52.0,0.0,15.0,71.48,32.0,25.0,1.0,102.0,101.0,2.0,20.0,0.051750,0.107914,0.030418,0.095890,0.111888,0.029630,0.715529,0.696612,0.770994,0.665529
1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,I have made sure my mother is comfortable with...,sup1,NaN,mum isn't speaking to me because I booked a fl...,ref,NaN,NaN,1,mum isn't speaking to me because I booked a fl...,0.368421,0.100000,0.210526,1.000000,1.000000,1.000000,0.046340,1.000000,71.48,32.0,25.0,1.0,102.0,101.0,2.0,20.0,77.57,24.0,19.0,1.0,74.0,71.0,0.0,14.0,0.095890,0.111888,0.029630,0.070015,0.136691,0.045455,0.696612,0.640240,0.665529,1.000000
2,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,mum isn't speaking to me because I booked a fl...,ref,NaN,Mum thought I was going to road trip with my b...,sup1,NaN,NaN,0,mum isn't speaking to me because I booked a fl...,1.000000,1.000000,1.000000,0.325581,0.000000,0.279070,1.000000,0.008724,77.57,24.0,19.0,1.0,74.0,71.0,0.0,14.0,97.20,38.0,32.0,4.0,136.0,132.0,1.0,27.0,0.070015,0.136691,0.045455,0.127093,0.154930,0.039568,0.640240,0.727418,1.000000,0.793905
3,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,reddit,relationships,Mother not speaking to me because of a trip ...,My boyfriend and I are long distance. We have ...,Mum thought I was going to road trip with my b...,sup1,NaN,Mum is mad at me for not flying on my own trip...,sup1,NaN,NaN,0,mum isn't speaking to me because I booked a fl...,0.325581,0.000000,0.279070,0.363636,0.181818,0.303030,0.008724,0.057345,97.20,38.0,32.0,4.0,136.0,132.0,1.0,27.0,97.54,17.0,16.0,1.0,53.0,52.0,0.0,15.0,0.127093,0.154930,0.039568,0.051750,0.107914,0.030418,0.727418,0.715529,0.793905,0.770994
4,LjvoXOAj5op3WqNnn5b7TZTG8mK7gM,batch3,train,t3_1zwek5,reddit,AskReddit,Can I sue my property management company and l...,My landlord left a falsified message taped to ...,My landlord is harassing me and my neighbours ...,sup1,NaN,landlord pretended to be another tenant and wr...,ref,NaN,NaN,1,landlord pretended to be another tenant and wr...,0.137931,0.000000,0.137931,1.000000,1.000000,1.000000,0.007511,1.000000,70.63,48.0,35.0,2.0,157.0,155.0,4.0,26.0,48.81,47.0,28.0,2.0,145.0,143.0,7.0,17.0,0.120353,0.129213,0.030100,0.108381,0.113636,0.037931,0.803380,0.603939,0.538846,1.000000


In [12]:
validation_comparisons.head()

,worker,batch,split,id,source,subsource,title,text,summary_0,policy_0,note_0,summary_1,policy_1,note_1,confidence,choice,ref_summary,m0_rouge_1_f,m0_rouge_2_f,m0_rouge_l_f,m1_rouge_1_f,m1_rouge_2_f,m1_rouge_l_f,m0_bleu,m1_bleu,m0_flesch_reading_ease,m0_syllable_count,m0_lexicon_count,m0_sentence_count,m0_char_count,m0_letter_count,m0_polysyllab_count,m0_monosyllab_count,m1_flesch_reading_ease,m1_syllable_count,m1_lexicon_count,m1_sentence_count,m1_char_count,m1_letter_count,m1_polysyllab_count,m1_monosyllab_count,m0_compression_ratio,m0_jaccard_similarity_1,m0_jaccard_similarity_2,m1_compression_ratio,m1_jaccard_similarity_1,m1_jaccard_similarity_2,m0_text_summary_xfmr_similarity,m1_text_summary_xfmr_similarity,m0_ref_summary_xfmr_similarity,m1_ref_summary_xfmr_similarity
0,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,reddit,relationships,I [23F] have just come out of 8 year relations...,This is my first post so please be kind :)\n\n...,"Just came out of 8 year relationship, don't kn...",sup4_t0.7,Ok,"I never dated/flirted as an adult, now I'm not...",sup4_t0.7,Ok,6,1,Started a long-term relationship as a teenager...,0.270270,0.153846,0.270270,0.30000,0.150000,0.30000,0.114334,0.056586,94.15,27.0,22.0,2.0,88.0,83.0,1.0,19.0,94.15,27.0,22.0,2.0,92.0,86.0,1.0,19.0,0.135404,0.130435,0.051136,0.140373,0.090164,0.021858,0.581621,0.511620,0.540373,0.580023
1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,reddit,relationships,I [23F] have just come out of 8 year relations...,This is my first post so please be kind :)\n\n...,Don't know how to date/flirt/flirt with someon...,sup4_t0.7,Repetitive,"Just came out of 8 year relationship, don't kn...",sup4_t0.7,Ok,7,1,Started a long-term relationship as a teenager...,0.205128,0.100000,0.205128,0.27027,0.153846,0.27027,0.042085,0.114334,67.76,35.0,24.0,2.0,124.0,118.0,1.0,14.0,94.15,27.0,22.0,2.0,88.0,83.0,1.0,19.0,0.182609,0.109244,0.039106,0.135404,0.130435,0.051136,0.641835,0.581621,0.475270,0.540373
2,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,reddit,relationships,I [23F] have just come out of 8 year relations...,This is my first post so please be kind :)\n\n...,Don't know how to date/flirt/flirt with someon...,sup4_t0.7,Repetitive,"I never dated/flirted as an adult, now I'm not...",sup4_t0.7,Ok,7,1,Started a long-term relationship as a teenager...,0.205128,0.100000,0.205128,0.30000,0.150000,0.30000,0.042085,0.056586,67.76,35.0,24.0,2.0,124.0,118.0,1.0,14.0,94.15,27.0,22.0,2.0,92.0,86.0,1.0,19.0,0.182609,0.109244,0.039106,0.140373,0.090164,0.021858,0.641835,0.511620,0.475270,0.580023
3,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,reddit,relationships,I [23F] have just come out of 8 year relations...,This is my first post so please be kind :)\n\n...,"Just came out of 8 year relationship, don't kn...",sup4_t0.7,Ok,Started a long-term relationship as a teenager...,ref,Ok,4,0,Started a long-term relationship as a teenager...,0.270270,0.153846,0.270270,1.00000,1.000000,1.00000,0.114334,1.000000,94.15,27.0,22.0,2.0,88.0,83.0,1.0,19.0,86.71,26.0,20.0,2.0,87.0,82.0,1.0,16.0,0.135404,0.130435,0.051136,0.131677,0.151786,0.057471,0.581621,0.633669,0.540373,1.000000
4,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,reddit,relationships,I [23F] have just come out of 8 year relations...,This is my first post so please be kind :)\n\n...,Don't know how to date/flirt/flirt with someon...,sup4_t0.7,Repetitive,Started a long-term relationship as a teenager...,ref,Ok,6,1,Started a long-term relationship as a teenager...,0.205128,0.100000,0.205128,1.00000,1.000000,1.00000,0.042085,1.000000,67.76,35.0,24.0,2.0,124.0,118.0,1.0,14.0,86.71,26.0,20.0,2.0,87.0,82.0,1.0,16.0,0.182609,0.109244,0.039106,0.131677,0.151786,0.057471,0.641835,0.633669,0.475270,1.000000


In [13]:
test_axis.head()

,worker,batch,split,id,source,subsource,title,text,summary,policy,note,compatible,accuracy,coverage,coherence,overall,ref_summary,m_rouge_1_f,m_rouge_2_f,m_rouge_l_f,m_bleu,m_flesch_reading_ease,m_syllable_count,m_lexicon_count,m_sentence_count,m_char_count,m_letter_count,m_polysyllab_count,m_monosyllab_count,m_compression_ratio,m_jaccard_similarity_1,m_jaccard_similarity_2,m_text_summary_xfmr_similarity,m_ref_summary_xfmr_similarity
0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,cnn_dailymail,dailymail,Newcastle United midfielder Moussa Sissoko fac...,Newcastle stand-in skipper Moussa Sissoko is f...,Moussa Sissoko was sent off against Liverpool ...,ref,"Misleading: ""Carver admits he is only concerne...",False,5,4,2.0,3,Moussa Sissoko was sent off against Liverpool ...,1.000000,1.000000,1.000000,1.000000,76.42,85.0,59.0,5.0,288.0,276.0,5.0,39.0,0.164449,0.170213,0.021127,0.735781,1.000000
1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,cnn_dailymail,dailymail,Newcastle United midfielder Moussa Sissoko fac...,Newcastle stand-in skipper Moussa Sissoko is f...,Newcastle have a mountain to climb in the Prem...,sup4_t0.7,"""Carver is determined to make the most of his ...",False,3,1,1.0,1,Moussa Sissoko was sent off against Liverpool ...,0.246575,0.025000,0.191781,0.005821,72.50,32.0,24.0,1.0,106.0,104.0,2.0,18.0,0.061312,0.070485,0.002532,0.459292,0.372433
2,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,cnn_dailymail,dailymail,Newcastle United midfielder Moussa Sissoko fac...,Newcastle stand-in skipper Moussa Sissoko is f...,Moussa Sissoko has been sent off following a d...,sup4_ppo_rm4_t.7,"""Carver feels the player should have been sent...",False,4,2,2.0,2,Moussa Sissoko was sent off against Liverpool ...,0.225000,0.047059,0.225000,0.008955,64.20,48.0,31.0,2.0,160.0,156.0,4.0,18.0,0.090304,0.120536,0.033419,0.653757,0.896054
3,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,cnn_dailymail,dailymail,Newcastle United midfielder Moussa Sissoko fac...,Newcastle stand-in skipper Moussa Sissoko is f...,Moussa Sissoko is facing a disciplinary action...,pretrain_xl_t.7,Doesnt summarize important points. Match info ...,False,7,3,6.0,3,Moussa Sissoko was sent off against Liverpool ...,0.257143,0.080000,0.228571,0.009180,52.19,30.0,19.0,1.0,90.0,90.0,1.0,9.0,0.051331,0.081818,0.034574,0.520803,0.638467
4,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,cnn_dailymail,dailymail,Newcastle United midfielder Moussa Sissoko fac...,Newcastle stand-in skipper Moussa Sissoko is f...,Newcastle need to start helping themselves now...,sup4_6b_t0.7,Completely misses main info.,False,7,1,3.0,1,Moussa Sissoko was sent off against Liverpool ...,0.061538,0.000000,0.061538,0.000683,74.19,20.0,14.0,1.0,80.0,77.0,1.0,9.0,0.044202,0.067873,0.021108,0.385909,0.180402


In [14]:
validation_axis.head()

,worker,batch,split,id,source,subsource,title,text,summary,policy,note,compatible,accuracy,coverage,coherence,overall,ref_summary,m_rouge_1_f,m_rouge_2_f,m_rouge_l_f,m_bleu,m_flesch_reading_ease,m_syllable_count,m_lexicon_count,m_sentence_count,m_char_count,m_letter_count,m_polysyllab_count,m_monosyllab_count,m_compression_ratio,m_jaccard_similarity_1,m_jaccard_similarity_2,m_text_summary_xfmr_similarity,m_ref_summary_xfmr_similarity
0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,reddit,relationship_advice,19f with fiance 20m and roommate 19m- fiance a...,"Recently, my fiance (20 m) and I (19f) moved ...",Fiance and I recently got infected with scabie...,sup4_ppo_rm4_t.7,'our apartment will not go away. I'm afraid he...,False,5.0,6.0,5.0,5.0,"infestation of scabies mites in apartment, roo...",0.173913,0.000000,0.130435,0.006990,87.92,46.0,35.0,4.0,150.0,144.0,3.0,27.0,0.229141,0.196429,0.085106,0.798759,0.626239
1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,reddit,relationship_advice,19f with fiance 20m and roommate 19m- fiance a...,"Recently, my fiance (20 m) and I (19f) moved ...",my fiance and I refuse to treat our room mate'...,pretrain_6b_t.7,NaN,False,4.0,4.0,7.0,4.0,"infestation of scabies mites in apartment, roo...",0.055556,0.000000,0.055556,0.007266,78.93,31.0,26.0,1.0,110.0,108.0,1.0,22.0,0.168120,0.148148,0.061453,0.554089,0.311793
2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,reddit,relationship_advice,19f with fiance 20m and roommate 19m- fiance a...,"Recently, my fiance (20 m) and I (19f) moved ...","fiancé and I contracted scabies, roommate refu...",sup4_6b_ppo_rm4_6b_t.7,the question s missing but the summary is good,False,6.0,6.0,7.0,6.0,"infestation of scabies mites in apartment, roo...",0.232558,0.040000,0.186047,0.014123,70.63,50.0,35.0,2.0,158.0,155.0,4.0,25.0,0.239103,0.235849,0.123596,0.902718,0.820978
3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,reddit,relationship_advice,19f with fiance 20m and roommate 19m- fiance a...,"Recently, my fiance (20 m) and I (19f) moved ...","fiance and I are infected with scabies, room m...",sup4_6b_t0.7,"a small inaccuracy and omission, otherwise good",False,6.0,6.0,7.0,6.0,"infestation of scabies mites in apartment, roo...",0.108108,0.000000,0.054054,0.009849,73.51,30.0,23.0,1.0,90.0,87.0,2.0,18.0,0.139477,0.179245,0.050000,0.808535,0.658740
4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,reddit,relationship_advice,19f with fiance 20m and roommate 19m- fiance a...,"Recently, my fiance (20 m) and I (19f) moved ...","Fiance and I contracted scabies, roommate refu...",sup4_12b_t0.7,"a small inaccuracy, otherwise good",False,5.0,7.0,7.0,6.0,"infestation of scabies mites in apartment, roo...",0.190476,0.044444,0.142857,0.016276,89.45,38.0,29.0,4.0,123.0,117.0,2.0,22.0,0.188045,0.169643,0.070270,0.819988,0.669971
